# Radio Record
This file contains functions to record radio directly from live internet stream; three example stream heads
are provided

In [1]:
# Requirement: need ffmpeg installed
# Windows: builds on https://www.gyan.dev/ffmpeg/builds/ - download link should be:
#   https://www.gyan.dev/ffmpeg/builds/ffmpeg-release-full.7z
#   Then, copy the ffmpeg installation to C:\Program Files\ffmpeg and  
#     set C:\Program Files\ffmpeg\bin in your PATH environemnt variables 
# Linux: sudo apt install ffmpeg

# Requirement: Google Cloud API access
# get google cloud account, enable the API for speech and translation, and install python library
# more info: https://cloud.google.com/python/getting-started
# create project and enable APIs: https://console.cloud.google.com/projectselector2/home
# under APIs & Services, credentials, create [additional] auth keys and download as JSON files
# pip install google-cloud-speech
# pip install google-cloud-translate

# Requirement: SpeechRecognition is a python library that can use a variety of speech recognition engines, including Google Cloud SpeechRecognition
# pip install SpeechRecognition

# Requirement: pydub allows us to record and manipulate sound files
# pip install pydub

import datetime
import os 
from urllib.request import urlopen
import speech_recognition as sr 
import json
from pydub import AudioSegment # uses FFMPEG
from pathlib import Path
from google.cloud import translate_v2 as translate
import six


In [2]:
# GOOGLE CLOUD CREDENTIALS

# type in your own GOOGLE json cloud key filename here that you got from 
# enabling your Google Cloud account and enabling speech to text and translation functions
# in your Google Cloud Projects page

# speech to text and translation load the keys in different ways, so we'll need to load them 
# twice - once as a global variable here in this run file, and once as an environment variable
with open('./google_speech_api_key-############.json') as filekey:
    gkey = json.load(filekey)

GOOGLE_CLOUD_SPEECH_CREDENTIALS = json.dumps(gkey) # for speech recognition
GOOGLE_APPLICATION_CREDENTIALS = "./google_speech_api_key-############.json" # for translation
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'google_speech_api_key-############.json'

In [3]:
# Define a function record that streams an internet audio stream into 
# a sound file for the duration specified 
def record(filepath, stream, duration):
    """records an internet audio stream for a given duration into a file"""
    fd = open(filepath, 'wb')
    begin = datetime.datetime.now()
    print("record beginning: " + str(begin))
    duration = datetime.timedelta(milliseconds=duration)
    while datetime.datetime.now() - begin < duration:
        data = stream.read(10000)
        fd.write(data)
    fd.close()
    print("record ending:    " + str(datetime.datetime.now()))
    print("filename: " + filepath)

In [5]:
# SPEECH RECOGNITION function
def do_google_speech_recognition(filename_wav):
    """wrapper function to run the google cloud speech to text"""

    s = ""  # declare s as a string

    speechrec = sr.Recognizer()  # instantiate a google speech recognizer

    try:
        with sr.AudioFile(filename_wav) as source: 
            speechrec.adjust_for_ambient_noise(source)
            audio = speechrec.record(source)
            s = speechrec.recognize_google_cloud(audio, 
                language="fr", 
                credentials_json=GOOGLE_CLOUD_SPEECH_CREDENTIALS) 
            print("Google Cloud Speech thinks you said: ")
            print(s)

    except sr.UnknownValueError:
        print("Google Cloud Speech could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Cloud Speech service; {0}".format(e))

    return s

In [13]:
# TRANSLATION function
def do_google_translation(input_text):
    """given a text in French, translate it to English using Google Cloud Translate"""

    # more on google cloud translate v2 here: 
    # https://github.com/googleapis/python-translate/blob/master/google/cloud/translate_v2/client.py 

    # instantiate the client
    translate_client = translate.Client()

    # error check on binary typing
    if isinstance(input_text, six.binary_type):
        input_text = input_text.decode("utf-8")
        print("decoding to utf-8")

    # if you want to download and print supported languages, uncomment and run this:

    #supported_languages = translate_client.get_languages()
    #for language in supported_languages:
    #    print(u"{name} ({language})".format(**language))

    result = translate_client.translate(input_text, target_language="en", source_language="fr")

    #print(u"Original Text: {}".format(result["input"]))
    #print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))
    print(u"Translation: {}".format(result["translatedText"]))
    
    return str( result["translatedText"] )

# Run blocks
Here is sample code to record snippets of internet radio; then using Google Cloud, we can 
do speech to text recognition, and translation, at least for French (use RFI Afrique)

In [10]:
# Here we record a file

filename = datetime.datetime.now().strftime("stream-%Y%m%d-%H%M%S.mp3")
seconds_to_record = 30

# uncomment the line below for radiostudio Kalangou
#record(filename, urlopen("https://studiokalangou.orientedhosting.com/kalangou"), seconds_to_record *1000) 

# uncomment the line below for radio Sahara  https://www.radioline.co/listen-to-radio-sahara
#record(filename, urlopen("http://www.radioking.com/play/radio-sahara?_=1454863441"), seconds_to_record *1000) 

# uncomment the line below for radio RFI Afrique https://www.rfi.fr/fr/afrique/ 
record(filename, urlopen("https://rfiafrique64k.ice.infomaniak.ch/rfiafrique-64.mp3"), seconds_to_record *1000) 


record beginning: 2021-01-24 20:13:57.675913
record ending:    2021-01-24 20:14:27.766576
filename: stream-20210124-201356.mp3


In [11]:
# Here we do google speech recognition
# if you have problems it's most likely your key

# convert mp3 file temporarily to a wav file
filename_wav = filename + ".tmp.wav"
sound_segment = AudioSegment.from_mp3(filename) # load the mp3
sound_segment = sound_segment.set_channels(1) # kock down to 1 channel to save space hopefully
sound_segment.export(filename_wav, format="wav") # write to wav file

# get the text using the google speech recognition api
spoken_text = do_google_speech_recognition(filename_wav)

# delete the temporary wav file
os.remove(filename_wav)

Google Cloud Speech thinks you said: 
oui parce que tu sais il y a il y a très peu de jeune congolais qui joue autre chose que la rumba donc c'est ses ces gens-là ils sont plus par rapport au public et et on les met plus dans ta catégorie à toi donc on a rigolé et j'ai bien de rencontrer des composés qui n'est pas là justement pourquoi vous avez discuté tous les deux éléments de pas suivre l'autoroute de la rumba qui passe au Congo dans mon cas moi en fait c'est je n'avais pas décidé de devenir musicien je suis allé au séminaire moi prêtre et 


In [14]:
# here we do the google translate to English
# if you have problems it's most likely your key

translated_text = do_google_translation(spoken_text)


Translation: yes because you know there are very few young Congolese who play anything other than rumba so these are the people they are more in relation to the public and we put them more in your category So we laughed and I had a good time meeting compounds that are not there precisely why you discussed both elements of not following the rumba highway that passes through Congo in my case me in fact it is I had not decided to become a musician I went to the seminary as a priest and
